In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Training import model, utils, dataset, train
import numpy as np
from sklearn import preprocessing
from IPython.display import HTML
from matplotlib import animation
from mpl_toolkits.mplot3d import axes3d

Load Models and Data

In [2]:
test_model = model.CSGainAndBandwidthManually()

# load data
data = np.array(utils.parseGainAndBWCsv('Data/BW_Gain2.0.csv')).astype(float)


In [3]:
print(data.shape)
print(data)

(668, 2, 2)
[[[ 1.000e-06  1.590e+03]
  [ 3.022e+00  9.660e+10]]

 [[ 1.000e-06  1.640e+03]
  [ 3.231e+00  9.330e+10]]

 [[ 1.000e-06  1.680e+03]
  [ 3.433e+00  9.030e+10]]

 ...

 [[ 1.000e-05  8.954e+02]
  [ 5.210e-01  6.070e+10]]

 [[ 1.000e-05  9.417e+02]
  [-8.350e-01  7.100e+10]]

 [[ 1.000e-05  9.881e+02]
  [-2.015e+00  8.400e+10]]]


Normalize data

In [4]:
data = data.reshape(data.shape[0],4)
feature = data.T[0]

# ensure norm and denorm work 
norm, data_min, data_max = utils.normalize(feature)
feature_1 = utils.denormalize(norm, data_min, data_max)


min_max = dict()
for i in range(4):
    feature = data.T[i]
    norm_feature, data_min, data_max = utils.normalize(feature)
    data.T[i] = norm_feature
    min_max[i] = [data_min, data_max]
data = data.reshape(data.shape[0],2,2)

In [5]:
# Create Dataset and split
data_x = data[:, 1]
data_y = data[: ,0]
dataset = dataset.CircuitSynthesisGainAndBandwidthManually(data_x, data_y)
train_dataset, val_dataset = utils.splitDataset(dataset, 0.8)

In [6]:
dtype = torch.FloatTensor
loss_fn = nn.L1Loss().type(dtype)  # loss can be changed here. This is the first one i tried that gave meaningful results
x = optim.Adagrad
model = test_model
optimizer = x(model.parameters(), lr=0.01)#0.06448295999961791)  # TODO haven't experimented with this yet

train_data = DataLoader(train_dataset, batch_size=5)
validation_data = DataLoader(val_dataset, batch_size=5)

In [ ]:
# train nn and check accuracy
acc, preds, idx = train.check_accuracy(model, validation_data, .1)
losses, accs, part_accs = train.trainWValidation(model, train_data, validation_data, loss_fn, optimizer, num_epochs=200, print_every=10)
# TODO accuracy may not be right. this was a quick attempt
#print(losses[-1])
#acc, preds = train.check_accuracy(test_model, validation_data, loss_fn)

Got 63 / 268 partially correct (23.51 pct)
Got 5 / 134 correct (3.73 pct)
Got 269 / 1068 partially correct (25.19 pct)
Got 33 / 534 correct (6.18 pct)
Got 75 / 268 partially correct (27.99 pct)
Got 10 / 134 correct (7.46 pct)
Got 255 / 1068 partially correct (23.88 pct)
Got 63 / 534 correct (11.80 pct)
Got 80 / 268 partially correct (29.85 pct)
Got 23 / 134 correct (17.16 pct)
Got 264 / 1068 partially correct (24.72 pct)
Got 81 / 534 correct (15.17 pct)
Got 79 / 268 partially correct (29.48 pct)
Got 26 / 134 correct (19.40 pct)
Got 265 / 1068 partially correct (24.81 pct)
Got 82 / 534 correct (15.36 pct)
Got 84 / 268 partially correct (31.34 pct)
Got 27 / 134 correct (20.15 pct)
Got 271 / 1068 partially correct (25.37 pct)
Got 95 / 534 correct (17.79 pct)
Got 86 / 268 partially correct (32.09 pct)
Got 30 / 134 correct (22.39 pct)
Got 282 / 1068 partially correct (26.40 pct)
Got 83 / 534 correct (15.54 pct)
Got 91 / 268 partially correct (33.96 pct)
Got 29 / 134 correct (21.64 pct)
Got 

### Final Accuracy check 

In [ ]:
acc, part_acc, preds = train.check_accuracy(model, validation_data, .1)
print(acc)
print()

### Plot Loss, accuracies

In [ ]:
# # plot losses per epoch
# plt.plot(acc)
# plt.show()
# plt.plot(accs)
# plt.show()
# plt.plot(part_accs)
# plt.show()

### Save Model if you want 

In [11]:
if False:
    torch.save(model.state_dict(), "best10pctmodel")

model.load_state_dict(torch.load("best10pctmodel"))
acc, part_acc, preds = train.check_accuracy(model, validation_data, .1)
print(acc)

Got 63 / 268 partially correct (23.51 pct)
Got 5 / 134 correct (3.73 pct)
0.03731343283582089


# Display Data to see where we fail

These take a miniute to run. 

## Normalized Bandwith (training data)

In [ ]:
preds = np.array(preds)
train_data = np.array(train_dataset)
print(preds.shape)
print(train_data.shape)

fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = train_data.T[1,0,:]
xline = train_data.T[0,1,:]
yline = train_data.T[0,0,:]
ax.set_zlabel('Gain')
ax.set_xlabel('Resistor Value')
ax.set_ylabel('Transistor Width')
ax.scatter3D(xline, yline, zline,c=zline, cmap = 'plasma')

def animate(frame):
    ax.view_init(30, frame/4)
    plt.pause(.001)
    return fig

anim = animation.FuncAnimation(fig, animate, frames=700, interval=25)
HTML(anim.to_html5_video())

## Predicted Bandwidth values

In [ ]:
train_data = np.array(train_dataset)
acc, part_acc, preds = train.check_accuracy(model, train_data, .1)
preds = np.array(preds)
print(preds.shape)
print(train_data.shape)

fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = preds.T[0,:]
xline = train_data.T[0,1,:]
yline = train_data.T[0,0,:]
ax.set_zlabel('Gain')
ax.set_xlabel('Resistor Value')
ax.set_ylabel('Transistor Width')
ax.scatter3D(xline, yline, zline,c=zline, cmap = 'plasma')

def animate(frame):
    ax.view_init(30, frame/4)
    plt.pause(.001)
    return fig

anim = animation.FuncAnimation(fig, animate, frames=700, interval=25)
HTML(anim.to_html5_video())

## Normalized Gain (training data)

In [ ]:
train_data = np.array(train_dataset)
print(preds.shape)
print(train_data.shape)

fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = train_data.T[1,1,:]
xline = train_data.T[0,1,:]
yline = train_data.T[0,0,:]
ax.set_zlabel('Gain')
ax.set_xlabel('Resistor Value')
ax.set_ylabel('Transistor Width')
ax.scatter3D(xline, yline, zline,c=zline, cmap = 'RdBu')

def animate(frame):
    ax.view_init(30, frame/4)
    plt.pause(.001)
    return fig

anim = animation.FuncAnimation(fig, animate, frames=500, interval=75)
HTML(anim.to_html5_video())

## Predicted Gain values

In [ ]:
preds = np.array(preds)
train_data = np.array(train_dataset)
print(preds.shape)
print(train_data.shape)

fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = preds.T[1,:]
xline = train_data.T[0,1,:]
yline = train_data.T[0,0,:]
ax.set_zlabel('Gain')
ax.set_xlabel('Resistor Value')
ax.set_ylabel('Transistor Width')
ax.scatter3D(xline, yline, zline,c=zline, cmap = 'RdBu')

def animate(frame):
    ax.view_init(30, frame/4)
    plt.pause(.001)
    return fig

anim = animation.FuncAnimation(fig, animate, frames=360, interval=10)
HTML(anim.to_html5_video())